In [193]:
import pandas as pd
import numpy as np
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error
import requests
from bs4 import BeautifulSoup
import re
import dateutil
import sys
import logging
from typing import List
from google.cloud import bigquery
from google.cloud import bigquery
from google.oauth2 import service_account

In [194]:
#---------------------Data Cleaning/Transformation-----------------------------------------
#read csv to dataframe
weather_df = pd.read_csv('./data/noaa_weather_data.csv', index_col='DATE')
#rename and filter columns which can be useful
core_weather_df = weather_df[['NAME','PRCP', 'SNOW','SNWD', 'TMAX','TMIN']].copy()
core_weather_df.columns = ['name','precip', 'snow', 'snow_depth', 'temp_max','temp_min']
#fill missing data points
core_weather_df['precip'] = core_weather_df['precip'].fillna(0)
core_weather_df['snow'] = core_weather_df['snow'].fillna(0)
core_weather_df['snow_depth'] = core_weather_df['snow_depth'].fillna(0)
#map the name to a number variable to be used as a predictor
core_weather_df['name_num'] = pd.factorize(core_weather_df['name'])[0]
#'ffill' ('forward fill') fills the value in with the value from the previous date this will apply to the temp_min/max columns since the others have already been filled in
core_weather_df = core_weather_df.fillna(method='ffill')
#change date index to a datetime data type
core_weather_df.index = pd.to_datetime(core_weather_df.index)
#make a visual representation of the data and remove cities that have large chunks of missing data
core_weather_df = core_weather_df[core_weather_df.name.isin(['BEMIDJI, MN US', 'SHARJAH INTER. AIRP, AE']) == False]
# core_weather_df.groupby(by='name').plot()
#create a column 'target_temp_max' by shifting all values in the temp_max column back a day...creates a column based on temperatures from tomorrows temps
core_weather_df['target_temp_max_day_1'] = core_weather_df.groupby('name')['temp_max'].shift(-1)
core_weather_df['target_temp_max_day_2'] = core_weather_df.groupby('name')['target_temp_max_day_1'].shift(-1)
core_weather_df['target_temp_max_day_3'] = core_weather_df.groupby('name')['target_temp_max_day_2'].shift(-1)
core_weather_df['target_temp_max_day_4'] = core_weather_df.groupby('name')['target_temp_max_day_3'].shift(-1)
core_weather_df['target_temp_max_day_5'] = core_weather_df.groupby('name')['target_temp_max_day_4'].shift(-1)
core_weather_df['target_temp_max_day_6'] = core_weather_df.groupby('name')['target_temp_max_day_5'].shift(-1)
core_weather_df['target_temp_max_day_7'] = core_weather_df.groupby('name')['target_temp_max_day_6'].shift(-1)
core_weather_df['target_temp_max_day_8'] = core_weather_df.groupby('name')['target_temp_max_day_7'].shift(-1)
core_weather_df['target_temp_max_day_9'] = core_weather_df.groupby('name')['target_temp_max_day_8'].shift(-1)
core_weather_df['target_temp_max_day_10'] = core_weather_df.groupby('name')['target_temp_max_day_9'].shift(-1)
#remove the last row since its a NaN value cuz that value would be in the future
core_weather_df = core_weather_df.dropna()
#training set includes all data before and including dec 12th 2021, this is teh set that is used to train the model to predict future data
training_set = core_weather_df.loc[:'2021-12-31']
#test set includes all data after and including jan 1st 2022
test_set = core_weather_df.loc['2022-01-01':]

In [195]:
#-----------------create/test data model--------------------------------
#using Ridge regression to minimize overfitting
regression = Ridge(alpha=.1)
predictors = ['precip','temp_max','temp_min']
#training set includes all data before and including dec 12th 2021, this is teh set that is used to train the model to predict future data
training_set = core_weather_df.loc[:'2021-12-31']
# #test set includes all data after and including jan 1st 2022
test_set = test_set.loc['2022-01-01':]

# test_set_shifted = test_set.shift(365, freq='D')
# test_set2= test_set_shifted.loc['2023-01-01':'2023-12-31']

#train the model based on the predictors
regression.fit(training_set[predictors], training_set['target_temp_max_day_1'])
predictions = regression.predict(test_set[predictors])

combined_df = pd.concat([test_set[['name','target_temp_max_day_1']], pd.Series(predictions, index = test_set.index)], axis=1)
combined_df.columns = ['name','actual','Predicted_temp_next_day']

core_weather_df.name.unique()
display(combined_df.tail())

,name,actual,Predicted_temp_next_day
DATE,,,
2022-12-27,"ORLANDO EXECUTIVE AIRPORT, FL US",74.0,63.781268
2022-12-28,"ORLANDO EXECUTIVE AIRPORT, FL US",80.0,72.508085
2022-12-29,"ORLANDO EXECUTIVE AIRPORT, FL US",80.0,78.633212
2022-12-30,"ORLANDO EXECUTIVE AIRPORT, FL US",76.0,78.919998
2022-12-31,"ORLANDO EXECUTIVE AIRPORT, FL US",80.0,76.748487
2023-01-01,"ORLANDO EXECUTIVE AIRPORT, FL US",82.0,79.923749
2023-01-02,"ORLANDO EXECUTIVE AIRPORT, FL US",82.0,80.722719
2023-01-03,"ORLANDO EXECUTIVE AIRPORT, FL US",83.0,81.152898
2023-01-04,"ORLANDO EXECUTIVE AIRPORT, FL US",77.0,83.416492


In [196]:
#------------scrape current data an append to historical dataset---------------------------------
#-------------------Extract-------------------------
urls = ["https://forecast.weather.gov/MapClick.php?lat=57.0826&lon=-135.2692#.Y-vs_9LMJkg",
        'https://forecast.weather.gov/MapClick.php?lat=45.5118&lon=-122.6756#.Y-vtHNLMJkg',
        'https://forecast.weather.gov/MapClick.php?lat=35.0842&lon=-106.649#.ZAKIR9LMJhE',
        'https://forecast.weather.gov/MapClick.php?lat=44.8017&lon=-68.7708#.ZAKIY9LMJhE',
        'https://forecast.weather.gov/MapClick.php?lat=64.8453&lon=-147.7221#.ZAKId9LMJhE',
        'https://forecast.weather.gov/MapClick.php?lat=20.8986&lon=-156.4305#.ZAKIjtLMJhE',
        'https://forecast.weather.gov/MapClick.php?lat=48.1786&lon=-114.3037#.ZAKIqNLMJhE',
        'https://forecast.weather.gov/MapClick.php?lat=37.4142&lon=-79.143#.ZAKIwtLMJhE',
        'https://forecast.weather.gov/MapClick.php?lat=42.9467&lon=-87.8967#.ZAKI0tLMJhE',
        'https://forecast.weather.gov/MapClick.php?lat=28.4272&lon=-81.308#.ZAKI5NLMJhE',
        'https://forecast.weather.gov/MapClick.php?lat=39.8784&lon=-75.2402#.ZAKJAdLMJhE'
        ]
        
combined_df = pd.DataFrame()

for url in urls:
    r = requests.get(url)
    soup = BeautifulSoup(r.content,"html.parser")

    #various containers
    item1 = soup.find_all(id='current_conditions-summary')
    item2 = soup.find_all(id='current_conditions_detail')
    item4 = soup.find_all(id='tombstone-container')

    #raw data
    temp_f = [item.find(class_="myforecast-current-lrg").get_text() for item in item1]
    temp_min = soup.find('p', {'class': 'temp temp-low'}).text.strip()
    temp_max = soup.find('p', {'class': 'temp temp-high'}).text.strip()


    #df of temperatures
    df_temperature = pd.DataFrame({"temp" : temp_f,'tempmin': temp_min,'tempmax': temp_max})

    #df_2 is a df of current conditions in detail (Humidity, Wind Speed, Barometer, Dewpoint, Visibility, Last update)
    table = soup.find_all('table')
    df_2 = pd.read_html(str(table))[0]
    df_2 = df_2.pivot(columns=0, values=1).ffill().dropna().reset_index().drop(columns=['index'])

    #merge both dataframes
    temp_df=pd.concat([df_temperature,df_2],axis=1)

    #scrape lattitude, longitude, and elevation 
    lat_lon_elev = soup.find('span', {'class': 'smallTxt'}).text.strip()
    lat, lon, elev = re.findall(r'[-+]?\d*\.\d+|\d+', lat_lon_elev)

    #scrape name
    station = soup.find('h2', {'class': 'panel-title'}).text.strip()

    #add location, lat, long, and elev to source_df
    temp_df['elevation_ft'] = elev
    temp_df['latitude'] = lat
    temp_df['longitude'] = lon
    temp_df['weather_station'] = station

    combined_df = pd.concat([temp_df, combined_df], ignore_index=True, sort=False)

display(combined_df)

,temp,tempmin,tempmax,Barometer,Dewpoint,Humidity,Last update,Visibility,Wind Chill,Wind Speed,elevation_ft,latitude,longitude,weather_station,Heat Index
0,41°F,Low: 42 °F⇑,High: 51 °F,29.77 in (1007.9 mb),38°F (3°C),89%,3 Mar 7:54 pm EST,4.00 mi,33°F (1°C),E 16 mph,7,39.87,75.23,"Philadelphia, Philadelphia International Airpo...",NaN
1,78°F,Low: 69 °F,High: 88 °F,29.85 in (1010.5 mb),61°F (16°C),56%,3 Mar 7:53 pm EST,10.00 mi,NaN,S 12 mph,89,28.42,81.32,Orlando International Airport (KMCO),80°F (27°C)
2,36°F,Low: 28 °F,High: 45 °F,29.55 in (1001.4 mb),29°F (-2°C),76%,3 Mar 6:52 pm CST,10.00 mi,26°F (-3°C),N 16 G 25 mph,722,42.96,87.9,"Milwaukee, General Mitchell International Airp...",NaN
3,45°F,Low: 44 °F,High: 62 °F,29.46 in (997.6 mb),43°F (6°C),93%,3 Mar 7:54 pm EST,5.00 mi,40°F (4°C),NE 10 mph,935,37.32,79.21,"Lynchburg, Lynchburg Regional Airport (KLYH)",NaN
4,28°F,Low: 20 °F,High: 36 °F,29.75 in (1007.45 mb),17°F (-8°C),63%,03 Mar 06:20 PM MST,10.00 mi,20°F (-7°C),WSW 8 MPH,2972,48.30417,114.26361,"Kalispell, Glacier Park International Airport ...",NaN
5,74°F,Low: 65 °F,High: 79 °F,29.92 in (1013.5 mb),66°F (19°C),76%,3 Mar 2:54 pm HST,9.00 mi,NaN,NE 15 mph,52,20.89,156.44,"Kahului, Kahului Airport (PHOG)",NaN
6,-1°F,Low: -21 °F,High: 2 °F,30.29 in (1027.2 mb),-12°F (-24°C),59%,3 Mar 3:53 pm AKST,10.00 mi,-19°F (-28°C),W 13 mph,433,64.8,147.88,"Fairbanks, Fairbanks International Airport (PAFA)",NaN
7,15°F,Low: 17 °F,High: 28 °F,30.04 in (1017.6 mb),11°F (-12°C),84%,3 Mar 7:53 pm EST,10.00 mi,NaN,Calm,190,44.8,68.82,"Bangor, Bangor International Airport (KBGR)",NaN
8,50°F,Low: 24 °F,High: 59 °F,29.78 in (1007.2 mb),23°F (-5°C),35%,3 Mar 5:52 pm MST,10.00 mi,47°F (8°C),S 8 mph,5351,35.04,106.61,"Albuquerque, Albuquerque International Airport...",NaN
9,43°F,Low: 33 °F,High: 41 °F,29.88 in (1011.85 mb),37°F (3°C),81%,03 Mar 05:20 PM PST,10.00 mi,NaN,SE 3 MPH,20,45.59578,122.60917,"Portland, Portland International Airport (KPDX)",NaN


In [197]:
urls = [
        'https://www.localconditions.com/weather-philadelphia-pennsylvania/19019/past.php',
        'https://www.localconditions.com/weather-orlando-florida/32801/past.php',
        'https://www.localconditions.com/weather-milwaukee-wisconsin/53201/past.php',
        'https://www.localconditions.com/weather-lynchburg-virginia/24501/past.php',
        'https://www.localconditions.com/weather-kalispell-montana/59901/past.php',
        'https://www.localconditions.com/weather-kahului-hawaii/96732/past.php',
        'https://www.localconditions.com/weather-fairbanks-alaska/99701/past.php',
        'https://www.localconditions.com/weather-bangor-maine/04401/past.php',
        'https://www.localconditions.com/weather-albuquerque-new-mexico/87101/past.php',
        'https://www.localconditions.com/weather-portland-oregon/97201/past.php',
        "https://www.localconditions.com/weather-sitka-alaska/99835/past.php"
        ]

precip_df = pd.DataFrame()

for url in urls:
        r = requests.get(url)
        soup = BeautifulSoup(r.content,"html.parser")
        details = soup.select_one(".past_weather_express")
        # Find all div elements with class="panel"
        panel_divs = soup.find_all('div', {'class': 'panel'})
        # Extract the text content of each div element and store it in a list
        panel_texts = [panel_div.text.strip() for panel_div in panel_divs]
        # Print the list of extracted text content
        data = panel_texts[1]
        data = [item.strip() for item in data]
        data = [item for item in data if item]
        data = data[77:90]
        data="".join(data)
        df = pd.DataFrame([data], columns=['precip'])
        precip_df = pd.concat([precip_df, df], ignore_index=True, sort=False)
        
precip_df['precip'] = precip_df['precip'].str.extract(pat='(\d+\.?\d*)').astype(float)
precip_df = precip_df.fillna(0)

source_df =pd.concat([combined_df,precip_df],axis=1)
display(source_df.head(11))

,temp,tempmin,tempmax,Barometer,Dewpoint,Humidity,Last update,Visibility,Wind Chill,Wind Speed,elevation_ft,latitude,longitude,weather_station,Heat Index,precip
0,41°F,Low: 42 °F⇑,High: 51 °F,29.77 in (1007.9 mb),38°F (3°C),89%,3 Mar 7:54 pm EST,4.00 mi,33°F (1°C),E 16 mph,7,39.87,75.23,"Philadelphia, Philadelphia International Airpo...",NaN,0.001
1,78°F,Low: 69 °F,High: 88 °F,29.85 in (1010.5 mb),61°F (16°C),56%,3 Mar 7:53 pm EST,10.00 mi,NaN,S 12 mph,89,28.42,81.32,Orlando International Airport (KMCO),80°F (27°C),0.000
2,36°F,Low: 28 °F,High: 45 °F,29.55 in (1001.4 mb),29°F (-2°C),76%,3 Mar 6:52 pm CST,10.00 mi,26°F (-3°C),N 16 G 25 mph,722,42.96,87.9,"Milwaukee, General Mitchell International Airp...",NaN,0.000
3,45°F,Low: 44 °F,High: 62 °F,29.46 in (997.6 mb),43°F (6°C),93%,3 Mar 7:54 pm EST,5.00 mi,40°F (4°C),NE 10 mph,935,37.32,79.21,"Lynchburg, Lynchburg Regional Airport (KLYH)",NaN,0.101
4,28°F,Low: 20 °F,High: 36 °F,29.75 in (1007.45 mb),17°F (-8°C),63%,03 Mar 06:20 PM MST,10.00 mi,20°F (-7°C),WSW 8 MPH,2972,48.30417,114.26361,"Kalispell, Glacier Park International Airport ...",NaN,0.037
5,74°F,Low: 65 °F,High: 79 °F,29.92 in (1013.5 mb),66°F (19°C),76%,3 Mar 2:54 pm HST,9.00 mi,NaN,NE 15 mph,52,20.89,156.44,"Kahului, Kahului Airport (PHOG)",NaN,0.040
6,-1°F,Low: -21 °F,High: 2 °F,30.29 in (1027.2 mb),-12°F (-24°C),59%,3 Mar 3:53 pm AKST,10.00 mi,-19°F (-28°C),W 13 mph,433,64.8,147.88,"Fairbanks, Fairbanks International Airport (PAFA)",NaN,0.011
7,15°F,Low: 17 °F,High: 28 °F,30.04 in (1017.6 mb),11°F (-12°C),84%,3 Mar 7:53 pm EST,10.00 mi,NaN,Calm,190,44.8,68.82,"Bangor, Bangor International Airport (KBGR)",NaN,0.109
8,50°F,Low: 24 °F,High: 59 °F,29.78 in (1007.2 mb),23°F (-5°C),35%,3 Mar 5:52 pm MST,10.00 mi,47°F (8°C),S 8 mph,5351,35.04,106.61,"Albuquerque, Albuquerque International Airport...",NaN,0.006
9,43°F,Low: 33 °F,High: 41 °F,29.88 in (1011.85 mb),37°F (3°C),81%,03 Mar 05:20 PM PST,10.00 mi,NaN,SE 3 MPH,20,45.59578,122.60917,"Portland, Portland International Airport (KPDX)",NaN,0.039


In [198]:
#-----------Data Transformations-----------------

# Convert 'lat' and 'lon' columns to float type
source_df['latitude'] = source_df['latitude'].astype(float)
source_df['longitude'] = source_df['longitude'].astype(float)

# Convert 'elev' column to int type
source_df['elevation_ft'] = source_df['elevation_ft'].astype(int)

# Extract the numeric part of the temperature string and convert it to int
source_df['temp'] = source_df['temp'].str.extract('(\d+)').astype(float)

# Extract the numeric part of the tempmin string and convert it to int
source_df['tempmin'] = source_df['tempmin'].str.extract('(\d+)').astype(float)

# Extract the numeric part of the temperature string and convert it to int
source_df['tempmax'] = source_df['tempmax'].str.extract('(\d+)').astype(float)

# Split wind speed values into components and convert speed to int type
source_df['Wind Speed'] = source_df['Wind Speed'].str.extract('(\d+)', expand=False).fillna(0).astype(float)

# Convert 'humidity' column to int type
source_df['Humidity'] = source_df['Humidity'].str.extract('(\d+)', expand=False).astype(float)

# Convert 'barometer' column to float type, and convert inches to millibars
source_df['Barometer'] = round(source_df['Barometer'].apply(lambda x: float(x.split()[0]) * 33.8639 if 'in' in x and x != 'NA' else None), 2)

# Convert 'Visibility' column to float type
source_df['Visibility'] = source_df['Visibility'].str.extract('(\d+\.\d+|\d+)', expand=False).astype(float).round(2)

#Convert 'last_update' column to UTC
source_df['Last update'] = source_df['Last update'].apply(lambda x: dateutil.parser.parse(x, tzinfos={"EST": -5 * 3600, "CST": -6 * 3600, "MST": -7 * 3600,"PST": -8 * 3600,"AKST": -9 * 3600,"HST": -10 * 3600}))
source_df['Last update'] = source_df['Last update'].apply(lambda x: x.astimezone(dateutil.tz.tzutc()))
source_df['datetime'] = source_df['Last update'].dt.strftime('%Y-%m-%d')
source_df['datetime'] = pd.to_datetime(source_df['datetime'])

# make wind chill a float if exists and only display degree F
try:
    source_df[['Wind Chill']] = source_df['Wind Chill'].str.extract('(\d+)', expand=True).astype(float)
except:
    None

# extract the numeric value of dewpoint and only display the degree n farenheit
source_df[['Dewpoint']] = source_df['Dewpoint'].str.extract('(\d+)', expand=True).astype(float)

#change precip data type to float
source_df['precip'] = source_df['precip'].astype(float)

#rename weather station column to the city
def rename_station(value):
    if value == 'Portland, Portland International Airport (KPDX)':
        return 'PORTLAND INTERNATIONAL AIRPORT, OR US'
    elif value == 'Sitka - Sitka Airport (PASI)':
        return 'SITKA AIRPORT, AK US'
    elif value == 'Philadelphia, Philadelphia International Airport (KPHL)':
        return 'PHILADELPHIA INTERNATIONAL AIRPORT, PA US'
    elif value == 'Orlando International Airport (KMCO)':
        return 'ORLANDO EXECUTIVE AIRPORT, FL US'
    elif value == 'Milwaukee, General Mitchell International Airport (KMKE)':
        return 'MILWAUKEE MITCHELL AIRPORT, WI US'
    elif value == 'Lynchburg, Lynchburg Regional Airport (KLYH)':
        return 'LYNCHBURG REGIONAL AIRPORT, VA US'
    elif value == 'Kalispell, Glacier Park International Airport (KGPI)':
        return 'KALISPELL GLACIER AIRPORT, MT US'
    elif value == 'Kahului, Kahului Airport (PHOG)':
        return 'KAHULUI AIRPORT, HI US'
    elif value == 'Fairbanks, Fairbanks International Airport (PAFA)':
        return 'FAIRBANKS INTERNATIONAL AIRPORT, AK US'
    elif value == 'Albuquerque, Albuquerque International Airport (KABQ)':
        return 'ALBUQUERQUE INTERNATIONAL AIRPORT, NM US'
    elif value == 'Bangor, Bangor International Airport (KBGR)':
        return 'BANGOR INTERNATIONAL AIRPORT, ME US'
source_df['name'] = source_df['weather_station'].map(rename_station)

#change the names and order of columns to better fit the historical data
source_df = source_df.rename({'Humidity': 'humidity', 'Wind Speed': 'windspeed', 'Visibility': 'visibility','Wind Chill': 'windchill','Dewpoint':'dewpoint','tempmax':'temp_max','tempmin':'temp_min'}, axis=1) 
#this line only includes necesarry columns
source_df = source_df[['name','datetime','precip','temp_max','temp_min','temp','windchill','dewpoint','humidity','windspeed','visibility']]
source_df = source_df.fillna(0)
source_df = source_df.set_index(['datetime'])
source_df.index.names = ['DATE']

#combine source df and test set so the test set had the most current info
test_set_forecast = pd.concat([test_set, source_df])
test_set_forecast = test_set_forecast[['name','precip','temp_max','temp_min']]
display(test_set)

,name,precip,snow,snow_depth,temp_max,temp_min,name_num,target_temp_max_day_1,target_temp_max_day_2,target_temp_max_day_3,target_temp_max_day_4,target_temp_max_day_5,target_temp_max_day_6,target_temp_max_day_7,target_temp_max_day_8,target_temp_max_day_9,target_temp_max_day_10
DATE,,,,,,,,,,,,,,,,,
2022-01-01,"FAIRBANKS INTERNATIONAL AIRPORT, AK US",0.00,0.1,33.9,-7.0,-17.0,0,-16.0,-14.0,-19.0,-26.0,-6.0,-26.0,-30.0,2.0,3.0,7.0
2022-01-02,"FAIRBANKS INTERNATIONAL AIRPORT, AK US",0.00,0.0,33.1,-16.0,-47.0,0,-14.0,-19.0,-26.0,-6.0,-26.0,-30.0,2.0,3.0,7.0,12.0
2022-01-03,"FAIRBANKS INTERNATIONAL AIRPORT, AK US",0.00,0.0,33.1,-14.0,-45.0,0,-19.0,-26.0,-6.0,-26.0,-30.0,2.0,3.0,7.0,12.0,21.0
2022-01-04,"FAIRBANKS INTERNATIONAL AIRPORT, AK US",0.00,0.0,33.1,-19.0,-37.0,0,-26.0,-6.0,-26.0,-30.0,2.0,3.0,7.0,12.0,21.0,20.0
2022-01-05,"FAIRBANKS INTERNATIONAL AIRPORT, AK US",0.00,0.0,33.1,-26.0,-38.0,0,-6.0,-26.0,-30.0,2.0,3.0,7.0,12.0,21.0,20.0,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-02-10,"ORLANDO EXECUTIVE AIRPORT, FL US",0.21,0.0,0.0,84.0,66.0,12,84.0,72.0,71.0,77.0,82.0,85.0,87.0,71.0,84.0,84.0
2023-02-11,"ORLANDO EXECUTIVE AIRPORT, FL US",0.01,0.0,0.0,84.0,67.0,12,72.0,71.0,77.0,82.0,85.0,87.0,71.0,84.0,84.0,85.0
2023-02-12,"ORLANDO EXECUTIVE AIRPORT, FL US",0.02,0.0,0.0,72.0,54.0,12,71.0,77.0,82.0,85.0,87.0,71.0,84.0,84.0,85.0,87.0


In [199]:
#-----------------create/test data model--------------------------------
#using Ridge regression to minimize overfitting
regression = Ridge(alpha=.1)
predictors = ['precip','temp_max','temp_min']
#training set includes all data before and including dec 12th 2021, this is teh set that is used to train the model to predict future data
training_set = core_weather_df.loc[:'2021-12-31']
# #test set includes all data after and including jan 1st 2022
#test_set = test_set.loc['2022-01-01':]

# test_set_shifted = test_set.shift(365, freq='D')
# test_set2= test_set_shifted.loc['2023-01-01':'2023-12-31']

#train the model based on the predictors
regression.fit(training_set[predictors], training_set['target_temp_max_day_1'])
predictions = regression.predict(test_set_forecast[predictors])

combined_df = pd.concat([test_set_forecast[['name']], pd.Series(predictions, index = test_set_forecast.index)], axis=1)
combined_df.columns = ['name','Predicted_temp_next_day']

core_weather_df.name.unique()
display(combined_df.tail(50))


,name,Predicted_temp_next_day
DATE,,
2023-01-07,"ORLANDO EXECUTIVE AIRPORT, FL US",70.848758
2023-01-08,"ORLANDO EXECUTIVE AIRPORT, FL US",76.543706
2023-01-09,"ORLANDO EXECUTIVE AIRPORT, FL US",75.314557
2023-01-10,"ORLANDO EXECUTIVE AIRPORT, FL US",72.825565
2023-01-11,"ORLANDO EXECUTIVE AIRPORT, FL US",74.167413
2023-01-12,"ORLANDO EXECUTIVE AIRPORT, FL US",75.714042
2023-01-13,"ORLANDO EXECUTIVE AIRPORT, FL US",67.175950
2023-01-14,"ORLANDO EXECUTIVE AIRPORT, FL US",55.054450
2023-01-15,"ORLANDO EXECUTIVE AIRPORT, FL US",59.458861
